# LULC Classification with CNN

In [1]:
# IMPORTS AND RANDOM SEEDS
import rasterio
import rasterio.plot
from rasterio.mask import mask
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt

import geopandas as gpd
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from patchify import unpatchify
from patchify import patchify

RNG_SEED = 42
np.random.seed(RNG_SEED)
tf.random.set_seed(RNG_SEED)

In [2]:
# Optional - Used to play sound when slow cells are finished executing
from IPython.display import Audio

In [3]:
# QoL Settings
debug = 0
optional_visuals = 0
play_sounds = 1

In [4]:
patch_size = 128      # Size to split each training image into
patch_step = 32       # Amount to step when splitting up images for training (<size means some overlap)

percent_augment = 0.5 # Amount of the training data set to randomly augment (brightness, orientation)

epochs = 150          # Epochs of training for the CNN (early stopping is enabled)

## Data Import

In [5]:
full_image_path = "input/aerial_scaled.tif"  # Aerial raster of entire area to be classified
labels_aerial_path = "input/aerial_512.tif" # Aerial raster of just labelled data
labels_classes_path = "input/labels_raster_512.tif" # Class raster of labelled data

In [6]:
# Load Full Image Aerial
with rasterio.open(full_image_path) as src:
    full_image_raw = src.read()  # Read all bands
    full_image_meta = src.profile

# Load Classes Raster of Labelled Sample
with rasterio.open(labels_classes_path) as src:
    labels_classes = src.read(1)  # Read the first (or only) band
    labels_classes_meta = src.profile

# Load Aerial Raster of Labelled Sample
with rasterio.open(labels_aerial_path) as src:
    labels_aerial_raw = src.read()
    labels_aerial_meta = src.profile

# Normalize images
full_image = full_image_raw.astype(np.float32) / 255.0  # Normalize to 0-1 range
labels_aerial = labels_aerial_raw.astype(np.float32) / 255.0 

# Print shapes
print("Full Image shape:", full_image.shape)  # Should be (Bands, Height, Width)
print("Labels Aerial shape:", labels_aerial.shape)
print("Labels Classes shape:", labels_classes.shape)  # Should be (Height, Width)

Full Image shape: (3, 1280, 1536)
Labels Aerial shape: (3, 512, 512)
Labels Classes shape: (512, 512)


#### ***Optional:*** *Visualize full aerial and labelled data*

In [7]:
if optional_visuals:
    # Visualize full image
    rasterio.plot.show(full_image)

    # Visualize lablled aerial and classes
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(reshape_as_image(labels_aerial))
    ax[1].imshow(labels_classes)

## Data Preparation

### Split Label Image into Patches

In [7]:
# Patchify
patches_classes = patchify(labels_classes, (patch_size, patch_size), step=patch_step) # shape (num_patches, 3, patch_size, patch_size)
patches_aerial = patchify(labels_aerial, (3, patch_size, patch_size), step=patch_step)[0] # shape (num_patches, patch_size, patch_size)

# Reshape patches into one list
X = patches_aerial.reshape(-1, 3, patch_size, patch_size) 
X = np.transpose(X, (0, 2, 3, 1)) 

Y = patches_classes.reshape(-1, patch_size, patch_size)

num_patches = X.shape[0]

if debug:
    print(f"(debug) - X (input) shape: {X.shape}")
    print(f"(debug) - Y (labels) shape: {Y.shape}")

print(f"Split into {num_patches} patches of size {patch_size} x {patch_size}, with step {patch_step}")

Split into 169 patches of size 128 x 128, with step 32


#### ***Optional:*** *Visualize patches*

In [9]:
patches_to_show = 4 # How many patches to show (<20 recommended)

if optional_visuals:    
    shown = 0

    # Visulaize aerial raster and class raster patches next to each other
    while shown < min(patches_to_show, num_patches):    
        fig, ax = plt.subplots(1, 2, figsize=(10,10))
    
        ax[0].imshow(X[shown])
        ax[1].imshow(Y[shown])

        shown += 1

## Data Processing
### Convert Class Labels to Categorical

In [8]:
# 0 building, 1 road, 2 vegetation, 3 alley/sidewalk/pavement
num_classes = len(np.unique(Y))

# Y classes are 1-indexed, need them 0-indexed
Y_shifted = Y - 1

# One hot encode the labels
Y_one_hot = to_categorical(Y_shifted, num_classes=num_classes)

print(f"Y: Encoded {num_classes} classes") 

Y: Encoded 4 classes


### Split Training and Testing Data

In [9]:
# Split data set into training and test sets
X_train_orig, X_test, Y_train_orig, Y_test = train_test_split(X, Y_one_hot, test_size=0.2, random_state=RNG_SEED)
print(f"Train set - X: {X_train_orig.shape}, Y: {Y_train_orig.shape}")
print(f"Test set -  X: {X_test.shape},  Y: {Y_test.shape}")

Train set - X: (135, 128, 128, 3), Y: (135, 128, 128, 4)
Test set -  X: (34, 128, 128, 3),  Y: (34, 128, 128, 4)


### Data Augmentation

In [10]:
def augment(image, label, brightness_factor=0.5, flip_prob=0.5):
    # Random brightness
    image = tf.image.random_brightness(image, max_delta=brightness_factor)

    # Random horizontal flipping
    if tf.random.uniform([]) < flip_prob:
        image = tf.image.flip_left_right(image)

    return image, label

In [12]:
# Randomly select indices for augmentation
num_aug_samples = int(percent_augment * X_train_orig.shape[0])
augment_indices = np.random.choice(X_train_orig.shape[0], num_aug_samples, replace=False)

# Apply augmentation
X_aug = []
Y_aug = []
for idx in augment_indices:
    new_image, new_label = augment(X_train_orig[idx], Y_train_orig[idx])
    X_aug.append(np.array(new_image))  # Convert Tensor to NumPy
    Y_aug.append(np.array(new_label))

# Convert to NumPy and concatenate with original dataset
X_aug = np.array(X_aug)
Y_aug = np.array(Y_aug)

X_train = np.concatenate([X_train_orig, X_aug], axis=0)
Y_train = np.concatenate([Y_train_orig, Y_aug], axis=0)

print(f"Added {num_aug_samples} augmented images. New training total: {X_train.shape[0]}")

Added 67 augmented images. New training total: 202


## Build CNN Model


In [13]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.UpSampling2D((2,2)),  # Upsample back

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.UpSampling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.UpSampling2D((2,2)),

    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.UpSampling2D((2,2)),

    layers.Conv2D(4, (1,1), activation='softmax')  # Output 4 classes per pixel
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 16, 16, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_4 (UpSampling2D)       │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 16, 16, 128)         │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_5 (UpSampling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 32, 32, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_6 (UpSampling2D)       │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 64, 64, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_7 (UpSampling2D)       │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 128, 128, 4)         │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,365,924 (5.21 MB)

 Trainable params: 1,365,924 (5.21 MB)

 Non-trainable params: 0 (0.00 B)

### Fit the Model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Fit the model
history = model.fit(X_train, Y_train, 
                    validation_data=(X_test, Y_test),
                    epochs=epochs, batch_size=10, 
                    callbacks=[early_stopping],
                   )

Epoch 1/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - accuracy: 0.4861 - loss: 1.2297 - val_accuracy: 0.5164 - val_loss: 1.1278
Epoch 2/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.5252 - loss: 1.1245 - val_accuracy: 0.5147 - val_loss: 1.1040
Epoch 3/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.5253 - loss: 1.1045 - val_accuracy: 0.5164 - val_loss: 1.0929
Epoch 4/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.5253 - loss: 1.1239 - val_accuracy: 0.5164 - val_loss: 1.0841
Epoch 5/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.5253 - loss: 1.0829 - val_accuracy: 0.5164 - val_loss: 1.0650
Epoch 6/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.5253 - loss: 1.0721 - val_accuracy: 0.5164 - val_loss: 1.0032
Epoch 7/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.5262 - loss: 1.0237 - val_accuracy: 0.4865 - val_loss: 1.0715
Epoch 8/150
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.4945 - loss: 1.0631 - val_accu

### Evaluate Accuracy

In [ ]:
# Plot training loss & accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Accuracy")

plt.show()

# Beep when done, if enabled
if play_sounds:
    beep = np.sin(2*np.pi*800*np.arange(3000*2)/10000)
    Audio(beep, rate=6000, autoplay=True)

## Predict the Image

### Full Image Preprocessing

In [ ]:
patch_full_step = patch_step

full_image_reshaped = reshape_as_image(full_image)
print("Full Image shape:", full_image_reshaped.shape)  # (2347, 2560, 3)

patches_full = patchify(full_image_reshaped, (patch_size, patch_size, 3), step=patch_full_step)
#patches_full = patches_full[0]
patches_full_x, patches_full_y = patches_full.shape[0:2]
print(patches_full_x, patches_full_y)
print("Patched Image shape:", patches_full.shape)  

patches_full_reshaped = patches_full.reshape(-1, patch_size, patch_size, 3)
print("Reshaped for Model shape:", patches_full_reshaped.shape)  

### Full Image Prediction

In [ ]:
predictions = model.predict(patches_full_reshaped)
print("Predictions shape:", predictions.shape) 

predictions = predictions.reshape(patches_full.shape[0], patches_full.shape[1], patch_size, patch_size, -1)
print("Reshaped Predictions shape:", predictions.shape)

predicted_classes = np.argmax(predictions, axis=-1)
print("Predicted Classes shape:", predicted_classes.shape)

## BLENDING MATRIX IMPLEMENTATION
# Initialize accumulation and weight matrices
full_prediction = np.zeros((patches_full_x * patch_full_step + patch_size, 
                            patches_full_y * patch_full_step + patch_size), dtype=np.float32)
weight_matrix = np.zeros_like(full_prediction, dtype=np.float32)

# Blend patches
for i in range(patches_full_x):
    for j in range(patches_full_y):
        x_start = i * patch_full_step
        x_end = x_start + patch_size
        y_start = j * patch_full_step
        y_end = y_start + patch_size
        
        full_prediction[x_start:x_end, y_start:y_end] += predicted_classes[i, j]
        weight_matrix[x_start:x_end, y_start:y_end] += 1  # Track contributions

# Normalize by the number of times each pixel was predicted
full_prediction /= weight_matrix
full_prediction = full_prediction.astype(np.uint8)
print("Blended Full Prediction shape:", full_prediction.shape)

rasterio.plot.show(full_image)
rasterio.plot.show(full_prediction)

# ---------------------------[ 
# Beep when done, if enabled
if play_sounds:
    beep = np.sin(2*np.pi*800*np.arange(3000*2)/10000)
    Audio(beep, rate=6000, autoplay=True)